### LLM과 RAG를 사용하여 AI 챗봇을 구현해보기

#### 1. 사용환경 준비
이번 과제에서는 OpenAI 모델 혹은 Gemini 모델 중 원하시는 모델을 사용하시면 됩니다. 원하는 모델에 따른 사용 환경을 준비하겠습니다.

**Yejin kang:** OpenAI를 사용하였고, 환경 변수로 키를 넣어주었기 때문에 환경 변수의 존재 여부를 확인하는 코드를 추가하였습니다.

In [2]:
from google.colab import userdata

openai_api_key = userdata.get("OPENAI_API_KEY")  # Secrets에서 직접 가져오기

if openai_api_key:
    os.environ["OPENAI_API_KEY"] = openai_api_key
    print("OpenAI API Key가 성공적으로 설정되었습니다.")
else:
    print("환경 변수에 OPENAI_API_KEY가 설정되어 있지 않습니다.")

OpenAI API Key가 성공적으로 설정되었습니다.


#### 2. 모델 로드하기
OpenAI 모델 혹은 Gemini 모델을 로드하여 model에 저장하세요.

**Yejin Kang:** 모델은 gpt 4o를 사용하였습니다.

In [4]:
!pip install langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.9 MB/s eta 0:00:00


In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

# 모델 초기화
model = ChatOpenAI(model="gpt-4o")

#### 3. 문서 로드하기
langchain의 `PyPDFLoader` 를 이용하여 문서를 불러옵니다.

문서는 아래의 문서 중 원하는 문서 중 하나를 다운로드 받으시면 됩니다.

**Yejin Kang:** 문서는 인공지능산업최신동향_2024년11월호를 선택하였습니다. 3가지 예시 중 최신 인공지능 산업동향에 대한 정보가 가장 흥미로웠기 때문입니다.


In [6]:
# 구글 코랩을 통해 작업해본적이 없기 때문에 기본 작업 디렉토리 확인이 필요하였습니다.
# 현재 작업 디렉토리 확인
print(os.getcwd())

/content


In [7]:
# 파일을 업로드해서 사용하기 위한 코드를 추가하였습니다.
# 아래의 코드를 실행하면 파일 업로드 창이 열려 업로드 할 파일을 선택할 수 있습니다.
from google.colab import files

# 파일 업로드
uploaded = files.upload()

Saving 인공지능산업최신동향_2024년11월호.pdf to 인공지능산업최신동향_2024년11월호.pdf


In [8]:
# langchain-community를 사용하기 위해 설치를 먼저 진행하였습니다.
# 이건 LangChain 라이브러리의 확장 모듈입니다.
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36


In [9]:
# pypdf 설치도 함께 진행하였습니다.
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 14.3 MB/s eta 0:00:00


In [10]:
from langchain.document_loaders import PyPDFLoader

# PDF 파일 경로
file_path = "/content/인공지능산업최신동향_2024년11월호.pdf"

# PDF 파일 로드
loader = PyPDFLoader(file_path)

# 페이지 별 문서 로드
docs = loader.load()

#### 4. 문서 청크로 나누기
불러온 문서를 대상으로 아래 i, ii 청킹방법을 모두 수행하세요.

청킹을 완수하면 청킹된 내용을 상위 10개까지 출력하고, 각 청킹방식과 parameter의 뜻을 markdown으로 정리해주세요. 청킹된 내용을 출력하는 이유는, 각 청킹방식에 따른 결과물을 여러분들이 눈으로 직접 확인하셨으면 하는 바입니다!

In [17]:
# 1. CharacterTextSplitter

from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n\n",                   # 문단 구분자를 \n\n 기준으로 나눔
    chunk_size=100,                     # 한 청크의 최대 길이를 설정하는 파라미터
    chunk_overlap=10,                   # 청크 간 겹치는 길이. 문맥을 유지하기 위해 얼마나 중복해서 겹칠것인가를 지정
    length_function=len,                # 텍스트 길이를 계산하는 함수. 기본적으로 'len' 함수를 사용한다.
    is_separator_regex=False,           # separator를 정규식으로 처리할 지 여부
)

character_splits = text_splitter.split_documents(docs)

# 결과 출력
print("CharacterTextSplitter 결과 (상위 10개):")
for i, chunk in enumerate(character_splits[:10]):
    print(f"청크 {i + 1}: {chunk}")

CharacterTextSplitter 결과 (상위 10개):
청크 1: page_content='2024년 11월호' metadata={'source': '/content/인공지능산업최신동향_2024년11월호.pdf', 'page': 0}
청크 2: page_content='2024년 11월호
Ⅰ. 인공지능 산업 동향 브리프 1. 정책/법제    ▹ 미국 민권위원회, 연방정부의 얼굴인식 기술 사용에 따른 민권 영향 분석························1   ▹ 미국 백악관 예산관리국, 정부의 책임 있는 AI 조달을 위한 지침 발표·····························2   ▹ 유로폴, 법 집행에서 AI의 이점과 과제를 다룬 보고서 발간··············································3   ▹ OECD, 공공 부문의 AI 도입을 위한 G7 툴킷 발표··························································4   ▹ 세계경제포럼, 생성AI 시대의 거버넌스 프레임워크 제시····················································5  2. 기업/산업    ▹ CB인사이츠 분석 결과, 2024년 3분기 벤처 투자 31%가 AI 스타트업에 집중··············6   ▹ 메타, 동영상 생성AI 도구 ‘메타 무비 젠’ 공개···································································7   ▹ 메타, 이미지와 텍스트 처리하는 첫 멀티모달 AI 모델 ‘라마 3.2’ 공개···························8   ▹ 앨런AI연구소, 벤치마크 평가에서 GPT-4o 능가하는 성능의 오픈소스 LLM ‘몰모’ 공개····9   ▹ 미스트랄AI, 온디바이스용 AI 모델 ‘레 미니스트로’ 공개·····················

In [16]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


# 2. RecursiveCharacterTextSplitter 설정
recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,            # 한 청크의 최대 길이
    chunk_overlap=10,          # 청크 간 겹치는 길이
    length_function=len,       # 텍스트 길이를 계산하는 함수
    is_separator_regex=False,  # separator를 정규식으로 처리할지 여부
)

# split_documents 사용
recursive_splits = recursive_text_splitter.split_documents(docs)

# 결과 출력
print("\nRecursiveCharacterTextSplitter 결과 (상위 10개):")
for i, chunk in enumerate(recursive_splits[:10]):
    print(f"청크 {i + 1}: {chunk.page_content}")


RecursiveCharacterTextSplitter 결과 (상위 10개):
청크 1: 2024년 11월호
청크 2: 2024년 11월호
청크 3: Ⅰ. 인공지능 산업 동향 브리프 1. 정책/법제    ▹ 미국 민권위원회, 연방정부의 얼굴인식 기술 사용에 따른 민권 영향 분석························1
청크 4: ▹ 미국 백악관 예산관리국, 정부의 책임 있는 AI 조달을 위한 지침 발표·····························2   ▹ 유로폴, 법 집행에서 AI의 이점과
청크 5: AI의 이점과 과제를 다룬 보고서 발간··············································3   ▹ OECD, 공공 부문의 AI 도입을 위한 G7
청크 6: 도입을 위한 G7 툴킷 발표··························································4   ▹ 세계경제포럼, 생성AI 시대의
청크 7: 생성AI 시대의 거버넌스 프레임워크 제시····················································5  2. 기업/산업    ▹ CB인사이츠
청크 8: ▹ CB인사이츠 분석 결과, 2024년 3분기 벤처 투자 31%가 AI 스타트업에 집중··············6   ▹ 메타, 동영상 생성AI 도구 ‘메타 무비 젠’
청크 9: ‘메타 무비 젠’ 공개···································································7   ▹ 메타, 이미지와 텍스트
청크 10: 이미지와 텍스트 처리하는 첫 멀티모달 AI 모델 ‘라마 3.2’ 공개···························8   ▹ 앨런AI연구소, 벤치마크 평가에서 GPT-4o


두 방식의 차이는 문서를 청크로 나누는 기준과 과정의 차이에서 발생한다.
1. CharacterTextSplitter
- 동작 방식:
  - 지정된 separator를 기준으로 텍스트를 나눈 후, 나뉜

#### 5. 벡터 임베딩 생성
OpenAI 모델의 경우 OpenAIEmbeddings , Gemini 모델의 경우GoogleGenerativeAIEmbeddings 를 이용해 텍스트를 벡터로 변환할 벡터 임베딩을 생성하세요.

In [18]:
from langchain_openai import OpenAIEmbeddings

# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

#### 6. 벡터 스토어 생성
앞서 만든 벡터 임베딩과 청크된 문서를 활용하여 `FAISS` 벡터 스토어를 생성하세요.

In [21]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 61.1 MB/s eta 0:00:00


In [24]:
import faiss
from langchain_community.vectorstores import FAISS


vector_store = FAISS.from_documents(documents=splits, embedding=embeddings)

#### 7. FAISS를 Retriever로 변환
RAG 체인에서 사용할 수 있도록 FAISS를 retriever로 변환하세요.

In [25]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 1})

#### 8. 프롬프트 템플릿 정의
프롬프트 템플릿을 정의하세요.

In [26]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# 프롬프트 템플릿 정의
contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the question using only the following context."),
    ("user", "Context: {context}\\n\\nQuestion: {question}")
])

#### 9. RAG 체인 구성
LangChain의 모델과 프롬프트를 연결하여 RAG 체인을 구성하세요.

In [27]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# 프롬프트 템플릿 정의
contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the question using only the following context."),
    ("user", "Context: {context}\\n\\nQuestion: {question}")
])

class DebugPassThrough(RunnablePassthrough):
    def invoke(self, *args, **kwargs):
        output = super().invoke(*args, **kwargs)
        print("Debug Output:", output)
        return output
# 문서 리스트를 텍스트로 변환하는 단계 추가
class ContextToText(RunnablePassthrough):
    def invoke(self, inputs, config=None, **kwargs):  # config 인수 추가
        # context의 각 문서를 문자열로 결합
        context_text = "\n".join([doc.page_content for doc in inputs["context"]])
        return {"context": context_text, "question": inputs["question"]}

# RAG 체인에서 각 단계마다 DebugPassThrough 추가
rag_chain_debug = {
    "context": retriever,                    # 컨텍스트를 가져오는 retriever
    "question": DebugPassThrough()        # 사용자 질문이 그대로 전달되는지 확인하는 passthrough
}  | DebugPassThrough() | ContextToText()|   contextual_prompt | model


#### 10. 챗봇 구동 확인
질문에 응답하는 챗봇을 구동하여 질문해보세요.

같은 질문을 일반 chat gpt 혹은 Gemini에 질문해보고 답변을 비교해보고, 왜 RAG이 필요한지 간단히 markdown으로 서술해주세요.

질문 내용: (1)메타 무비 젠이 뭐야, (2)이거 언제 공개 된 내용이야

이전에는 GhatGPT가 학습되지 않은 내용에 대해서는 답변할수 없거나, 이상한 말을 지어서 한것 같았는데 요즘 GPT는 웹 서치를 해서 정보를 제공해서 그런지 이런 동향 같은 정보는 RAG에 입력하지 않고도 일반 chatGPT가 검색으로 안내해주는 것 같습니다. 오히려 RAG 로 학습시켜서 질문하였을 때 상대적으로 덜 자세한 정보를 안내받았습니다. RAG로 개인적인 일정내역 같은 것을 학습시켜서 AI 비서 역할로 쓰면 좋을 것 같습니다.



In [33]:
while True:  # 무한 루프 조건 추가
    print("========================")
    query = input("질문을 입력하세요: ")

    if query.lower() in ["exit", "quit"]:  # 종료 조건
        print("프로그램을 종료합니다.")
        break

    try:
        response = rag_chain_debug.invoke(query)  # 메서드 호출
        print("Final Response:")
        print(response.content)
    except AttributeError as e:
        print(f"오류 발생: response 객체에 'content' 속성이 없습니다. ({e})")
    except Exception as e:
        print(f"예기치 못한 오류 발생: {e}")

질문을 입력하세요: 메타가 공개한 동영상 생성 도구에 대해 알려줘
Debug Output: 메타가 공개한 동영상 생성 도구에 대해 알려줘
Debug Output: {'context': [Document(metadata={'source': '/content/인공지능산업최신동향_2024년11월호.pdf', 'page': 9}, page_content='메타, 동영상 생성AI 도구 ‘메타 무비 젠’ 공개n메타가 동영상 생성, 개인화 동영상 제작, 동영상 편집, 오디오 생성과 같은 기능을 지원하는 ‘메타 무비 젠’을 공개하고')], 'question': '\x08메타가 공개한 동영상 생성 도구에 대해 알려줘'}
Final Response:
메타가 공개한 동영상 생성 도구는 '메타 무비 젠'입니다. 이 도구는 동영상 생성, 개인화 동영상 제작, 동영상 편집, 오디오 생성 등의 기능을 지원합니다. 이를 통해 사용자는 보다 쉽게 다양한 동영상을 제작하고 편집할 수 있습니다.
질문을 입력하세요: 메타 무비 젠이 뭐야
Debug Output: 메타 무비 젠이 뭐야
Debug Output: {'context': [Document(metadata={'source': '/content/인공지능산업최신동향_2024년11월호.pdf', 'page': 1}, page_content='‘메타 무비 젠’ 공개···································································7   ▹ 메타, 이미지와 텍스트')], 'question': '메타 무비 젠이 뭐야'}
Final Response:
메타 무비 젠은 메타(구 페이스북)에서 개발한 새로운 콘텐츠 생성 플랫폼입니다. 이 플랫폼은 이미지와 텍스트 데이터를 활용하여 사용자에게 다양한 형태의 콘텐츠를 제공합니다. 메타 무비 젠은 사용자가 창의적인 콘텐츠를 쉽게 제작할 수 있도록 도와주는 도구로, 특히 이미지